In [1]:
import requests
import spacy
from spacy import displacy
import pandas as pd

import seaborn as sns
%matplotlib inline

In [2]:
nlp = spacy.load('en')

In [3]:
url = r'https://www.gutenberg.org/files/16/16-0.txt'
r = requests.get(url)

In [4]:
import re

chapter_start_indexes = []
for match in re.finditer('Chapter', r.text):
    chapter_start_indexes.append(match.start())
    
chapters = []
previous_start = 0
for start_ind in chapter_start_indexes:
    text = r.text[previous_start:start_ind]
    chapters.append(text)
    previous_start = start_ind

In [26]:
# observe the named entities tagged as PERSON
from collections import defaultdict

chapter_characters = {}
for chapter_num, chapter in enumerate(chapters):
    if chapter_num < 18:
        continue
    characters = defaultdict(int)
    for chunk in chapter.split('\r\n\r\n'):
        doc = nlp(chunk)
        for ent in doc.ents:
            if 'PERSON' in ent.label_:
                entity = ent.text.lower().strip()
                characters[entity] += 1
    chapter_characters[chapter_num] = characters
        

In [44]:
import pandas as pd
df = pd.DataFrame(chapter_characters)
df.fillna(0, inplace=True)
df['sum'] = df.sum(axis=1)
top_characters = df.sort_values('sum', ascending=False).index[0:5]
df.drop('sum', inplace=True, axis=1)
df = df[df.index.isin(top_characters)]

In [45]:
df.head()

,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
darling,27.0,37.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,15.0
john,11.0,9.0,16.0,33.0,0.0,13.0,6.0,4.0,0.0,5.0,7.0,0.0,0.0,9.0,1.0,13.0
michael,10.0,25.0,9.0,16.0,0.0,4.0,7.0,4.0,0.0,6.0,5.0,0.0,0.0,5.0,1.0,9.0
peter,5.0,4.0,51.0,31.0,20.0,45.0,13.0,33.0,13.0,21.0,29.0,8.0,21.0,3.0,29.0,7.0
wendy,16.0,7.0,33.0,12.0,2.0,17.0,12.0,17.0,2.0,13.0,13.0,1.0,4.0,6.0,4.0,13.0


In [46]:
df.reset_index().to_csv(r'pan.csv', index=False)

In [1]:
df = pd.read_csv(r'pan.csv')
#df.index = 'Unnamed: 0'
df.index = df['index']
df.drop('index', inplace=True, axis=1)
df.head()

NameError: name 'pd' is not defined

In [ ]:
sns.heatmap(df)